In [10]:
import string
import numpy as np
import PIL.Image

from os import listdir
from pickle import dump, load

from numpy import array
from numpy import argmax

from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers.merge import add
from keras.models import Model, load_model
from keras.layers import Input, Dense, LSTM, Embedding, Dropout
from keras.callbacks import ModelCheckpoint

In [11]:
def extract_features(filename):
    model = VGG16()
    model.layers.pop()
    model = Model(inputs=model.inputs, outputs=model.layers[-1].output)
    image = load_img(filename, target_size=(224, 224))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    feature = model.predict(image, verbose=0)
    return feature

def word_for_id(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None

def generate_desc(model, tokenizer, photo, max_length):
    in_text = 'startseq'
    for i in range(max_length):
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence], maxlen=max_length)
        yhat = model.predict([photo,sequence], verbose=0)
        print(yhat)
        yhat = argmax(yhat)
        word = word_for_id(yhat, tokenizer)
        if word is None:
            break
        in_text += ' ' + word
        if word == 'endseq':
            break
    return in_text

tokenizer = load(open('tokenizer.pkl', 'rb'))
max_length = 34

In [12]:
model = load_model('models/model_19.h5')
path = r'C:\Users\DELL\Downloads\Project\Image-captioning\Flicker8k_Dataset\47871819_db55ac4699.jpg'
photo = extract_features(path)

description = generate_desc(model, tokenizer, photo, max_length)
print(description)

[[2.0070108e-09 1.8112958e-09 8.5144593e-06 ... 1.9913728e-09
  1.2711884e-10 1.9518733e-09]]
[[1.5638268e-10 1.5574657e-10 3.1735493e-05 ... 1.5419771e-10
  1.3235459e-12 1.5577599e-10]]
[[1.2949221e-10 1.6763356e-10 1.5263242e-03 ... 1.3288859e-10
  2.2644541e-11 1.4316431e-10]]
[[3.6860584e-11 4.0932133e-11 7.0493094e-05 ... 3.7948134e-11
  9.3421678e-16 3.8892681e-11]]
[[5.4131133e-10 6.2895217e-10 3.1561276e-03 ... 6.1382377e-10
  8.3412335e-13 5.3085153e-10]]
[[1.9159145e-12 1.7784355e-12 3.5650632e-01 ... 1.6575860e-12
  9.5555809e-22 1.4898378e-12]]
startseq two men are playing soccer endseq
